In [1]:
import pandas as pd
import numpy as np
from utils import *

In [2]:
link1 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2017"
df1 = pd.read_html(link1, header=0)[2]
df2 = pd.read_html(link1, header=0)[3]
df3 = pd.read_html(link1, header=0)[4]
df4 = pd.read_html(link1, header=0)[5]

In [3]:
df_2017 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [4]:
df_2017.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,January,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3]
1,January,6,Arsenal,Lionsgate Premiere / Grindstone Entertainment ...,Steven C. Miller (director); Jason Mosberg (sc...,[4][5]
2,January,6,Between Us,IFC Films,Rafael Palacio Illingworth (director/screenpla...,[6]
3,January,13,Monster Trucks,Paramount Pictures / Paramount Animation / Nic...,Chris Wedge (director); Derek Connolly (screen...,[7][8]
4,January,13,The Bye Bye Man,STX Entertainment / Intrepid Pictures,Stacy Title (director); Jonathan Penner (scree...,[9]


In [5]:
df_2017.shape

(242, 6)

In [6]:
from tmdbv3api import TMDb
import json
import requests

In [7]:
tmdb = TMDb()
tmdb.api_key = "62520f0cb1e77a57e1c66e073f288bf6"

In [8]:
from tmdbv3api import Movie

In [9]:
tmdb_movie = Movie()

def getGenre(movie):
    genres = []
    movieId = tmdb_movie.search(movie)[0].id
    res = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}".format(movieId, tmdb.api_key))
    dataJson = res.json()
    if dataJson["genres"]:
        for i in range(len(dataJson["genres"])):
            genres.append(dataJson["genres"][i]["name"])
        return " ".join(genres)
    else:
        np.NaN
        
def getOverview(movie):
    movieId = tmdb_movie.search(movie)[0].id
    res = requests.get("https://api.themoviedb.org/3/movie/{}?api_key={}".format(movieId, tmdb.api_key))
    dataJson = res.json()
    if dataJson["overview"]:
        return dataJson["overview"]
    else:
        np.NaN

In [10]:
df_2017["genres"] = df_2017["Title"].map(lambda x: getGenre(str(x)))

In [11]:
df_2017["overview"] = df_2017["Title"].map(lambda x: getOverview(str(x)))

In [12]:
df_2017.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview
0,January,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3],Fantasy Action Thriller,Vampire death dealer Selene fends off brutal a...
1,January,6,Arsenal,Lionsgate Premiere / Grindstone Entertainment ...,Steven C. Miller (director); Jason Mosberg (sc...,[4][5],Action Crime Thriller,After the deadbeat brother of a businessman is...
2,January,6,Between Us,IFC Films,Rafael Palacio Illingworth (director/screenpla...,[6],Romance Adventure Drama,"Stranded after a tragic plane crash, two stran..."
3,January,13,Monster Trucks,Paramount Pictures / Paramount Animation / Nic...,Chris Wedge (director); Derek Connolly (screen...,[7][8],Action Comedy Science Fiction,Tripp is a high school senior with a knack for...
4,January,13,The Bye Bye Man,STX Entertainment / Intrepid Pictures,Stacy Title (director); Jonathan Penner (scree...,[9],Horror Thriller,When three college students move into an old h...


In [13]:
df_2017.isna().sum()

Opening               0
Opening.1             0
Title                 0
Production company    1
Cast and crew         0
Ref.                  8
genres                0
overview              0
dtype: int64

In [14]:
df_2017["Cast and crew"][0]

'Anna Foerster (director); Cory Goodman (screenplay); Kate Beckinsale, Theo James, Lara Pulver, James Faulkner, Charles Dance'

In [15]:
def getDirector(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2017["director"] = df_2017["Cast and crew"].apply(lambda x : getDirector(x))

In [17]:
df_2017.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview,director
0,January,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3],Fantasy Action Thriller,Vampire death dealer Selene fends off brutal a...,Anna Foerster
1,January,6,Arsenal,Lionsgate Premiere / Grindstone Entertainment ...,Steven C. Miller (director); Jason Mosberg (sc...,[4][5],Action Crime Thriller,After the deadbeat brother of a businessman is...,Steven C. Miller


In [18]:
def getActors(x):
    x_ = x.split(";")
    return x_[len(x_) - 1].split(",")[:3]

In [19]:
df_2017["actors"] = df_2017["Cast and crew"].apply(lambda x : getActors(x))

In [20]:
df_2017.head(2)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview,director,actors
0,January,6,Underworld: Blood Wars,Screen Gems / Lakeshore Entertainment / Sketch...,Anna Foerster (director); Cory Goodman (screen...,[2][3],Fantasy Action Thriller,Vampire death dealer Selene fends off brutal a...,Anna Foerster,"[ Kate Beckinsale, Theo James, Lara Pulver]"
1,January,6,Arsenal,Lionsgate Premiere / Grindstone Entertainment ...,Steven C. Miller (director); Jason Mosberg (sc...,[4][5],Action Crime Thriller,After the deadbeat brother of a businessman is...,Steven C. Miller,"[ Adrian Grenier, Johnathon Schaech, Nicolas..."


In [21]:
df_2017 = df_2017[["Title", "genres", "overview", "director", "actors"]]

In [22]:
df_2017["director"] = df_2017["director"].apply(lambda x : x.split(","))

In [23]:
df_2017["genres"] = df_2017["genres"].apply(lambda x : x.split())
df_2017["overview"] = df_2017["overview"].apply(lambda x : x.split())

In [24]:
df_2017.head()

,Title,genres,overview,director,actors
0,Underworld: Blood Wars,"[Fantasy, Action, Thriller]","[Vampire, death, dealer, Selene, fends, off, b...",[Anna Foerster],"[ Kate Beckinsale, Theo James, Lara Pulver]"
1,Arsenal,"[Action, Crime, Thriller]","[After, the, deadbeat, brother, of, a, busines...",[Steven C. Miller],"[ Adrian Grenier, Johnathon Schaech, Nicolas..."
2,Between Us,"[Romance, Adventure, Drama]","[Stranded, after, a, tragic, plane, crash,, tw...",[Rafael Palacio Illingworth],"[ Olivia Thirlby, Ben Feldman, Adam Goldberg]"
3,Monster Trucks,"[Action, Comedy, Science, Fiction]","[Tripp, is, a, high, school, senior, with, a, ...",[Chris Wedge],"[ Lucas Till, Jane Levy, Amy Ryan]"
4,The Bye Bye Man,"[Horror, Thriller]","[When, three, college, students, move, into, a...",[Stacy Title],"[ Douglas Smith, Doug Jones, Michael Trucco]"


In [25]:
df_2017["Tags"] = df_2017["overview"] + df_2017["genres"] + df_2017["actors"] + df_2017["director"]

In [26]:
df_2017 = df_2017[["Title", "Tags"]]

In [27]:
df_2017["Tags"] = df_2017["Tags"].apply(lambda x : " ".join(x))

In [28]:
df_2017.head()

,Title,Tags
0,Underworld: Blood Wars,Vampire death dealer Selene fends off brutal a...
1,Arsenal,After the deadbeat brother of a businessman is...
2,Between Us,"Stranded after a tragic plane crash, two stran..."
3,Monster Trucks,Tripp is a high school senior with a knack for...
4,The Bye Bye Man,When three college students move into an old h...


In [29]:
df_2017["Tags"] = df_2017["Tags"].apply(stem)

In [30]:
df_2017["Tags"] = df_2017["Tags"].apply(lambda x : x.lower())

In [31]:
df_2017.head()

,Title,Tags
0,Underworld: Blood Wars,vampir death dealer selen fend off brutal atta...
1,Arsenal,after the deadbeat brother of a businessman is...
2,Between Us,"strand after a tragic plane crash, two strange..."
3,Monster Trucks,tripp is a high school senior with a knack for...
4,The Bye Bye Man,when three colleg student move into an old hou...


In [32]:
df_2017.shape

(242, 2)

In [33]:
link1 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link1, header=0)[2]
df2 = pd.read_html(link1, header=0)[3]
df3 = pd.read_html(link1, header=0)[4]
df4 = pd.read_html(link1, header=0)[5]

In [34]:
df_2018 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [35]:
df_2018.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]


In [36]:
df_2018.shape

(269, 6)

In [37]:
df_2018.drop(df_2018[df_2018["Title"] == "La Boda de Valentina"].index)

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]
...,...,...,...,...,...,...
264,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,[163]
265,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[137]
266,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[226]
267,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",[258]


In [38]:
df_2018["genres"] = df_2018["Title"].map(lambda x: getGenre(str(x)))
df_2018["overview"] = df_2018["Title"].map(lambda x: getOverview(str(x)))

In [39]:
df_2018.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],Horror Mystery Thriller,Parapsychologist Elise Rainier and her team tr...
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],Thriller Drama,Mysterious events surround the travels of two ...
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],Action Thriller,A British Special Boat Service commando tracks...
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],Drama History Western,"It’s 1929 on the vast, desert-like, Eastern Ar..."
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],Action Thriller,"A businessman, on his daily commute home, gets..."


In [40]:
df_2018.isna().sum()

Opening               0
Opening.1             0
Title                 0
Production company    0
Cast and crew         0
Ref.                  0
genres                3
overview              0
dtype: int64

In [41]:
df_2018.dropna(inplace = True)

In [42]:
df_2018["Cast and crew"][0]

'Adam Robitel (director); Leigh Whannell (screenplay); Lin Shaye, Angus Sampson, Leigh Whannell, Spencer Locke, Caitlin Gerard, Kirk Acevedo, Bruce Davison'

In [43]:
def getDirector_(x):
    if "(director)" in x:
        x_ = x.split(" (director)")[0]
    elif "(director/screenplay)" in x:
        x_ = x.split(" (director/screenplay)")[0]
    elif "(directors/screenplay)" in x:
        x_ = x.split(" (directors/screenplay)")[0]
    elif "(director/\u200bscreenplay)" in x:
        x_ = x.split(" (director/\u200bscreenplay)")[0]
    else:
        x_ = x.split(" (directors)")[0]
    if len(x_.split(",")) > 0:
        return x_.split(",")
    else:
        return np.NaN

In [44]:
df_2018["director"] = df_2018["Cast and crew"].apply(lambda x : getDirector_(x))
df_2018["actors"] = df_2018["Cast and crew"].apply(lambda x : getActors(x))

In [45]:
df_2018.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview,director,actors
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2],Horror Mystery Thriller,Parapsychologist Elise Rainier and her team tr...,[Adam Robitel],"[ Lin Shaye, Angus Sampson, Leigh Whannell]"
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3],Thriller Drama,Mysterious events surround the travels of two ...,[Lauren Wolkstein],"[ Alex Pettyfer, James Freedson-Jackson, Emi..."
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4],Action Thriller,A British Special Boat Service commando tracks...,[Simon West],"[ Dominic Cooper, Austin Stowell, Gemma Chan]"
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5],Drama History Western,"It’s 1929 on the vast, desert-like, Eastern Ar...",[Warwick Thornton],"[ Bryan Brown, Sam Neill]"
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6],Action Thriller,"A businessman, on his daily commute home, gets...",[Jaume Collet-Serra],"[ Liam Neeson, Vera Farmiga, Patrick Wilson]"


In [46]:
df_2018 = df_2018[["Title", "genres", "overview", "director", "actors"]]
df_2018["genres"] = df_2018["genres"].apply(lambda x : x.split())
df_2018["overview"] = df_2018["overview"].apply(lambda x : x.split())

In [47]:
df_2018.head(2)

,Title,genres,overview,director,actors
0,Insidious: The Last Key,"[Horror, Mystery, Thriller]","[Parapsychologist, Elise, Rainier, and, her, t...",[Adam Robitel],"[ Lin Shaye, Angus Sampson, Leigh Whannell]"
1,The Strange Ones,"[Thriller, Drama]","[Mysterious, events, surround, the, travels, o...",[Lauren Wolkstein],"[ Alex Pettyfer, James Freedson-Jackson, Emi..."


In [48]:
df_2018["Tags"] = df_2018["overview"] + df_2018["genres"] + df_2018["actors"] + df_2018["director"]

In [49]:
df_2018 = df_2018[["Title", "Tags"]]
df_2018["Tags"] = df_2018["Tags"].apply(lambda x : " ".join(x))

In [50]:
df_2018.head()

,Title,Tags
0,Insidious: The Last Key,Parapsychologist Elise Rainier and her team tr...
1,The Strange Ones,Mysterious events surround the travels of two ...
2,Stratton,A British Special Boat Service commando tracks...
3,Sweet Country,"It’s 1929 on the vast, desert-like, Eastern Ar..."
4,The Commuter,"A businessman, on his daily commute home, gets..."


In [51]:
df_2018["Tags"] = df_2018["Tags"].apply(stem)

In [52]:
df_2018["Tags"] = df_2018["Tags"].apply(lambda x : x.lower())

In [53]:
df_2018.head()

,Title,Tags
0,Insidious: The Last Key,parapsychologist elis rainier and her team tra...
1,The Strange Ones,mysteri event surround the travel of two broth...
2,Stratton,a british special boat servic commando track d...
3,Sweet Country,"it’ 1929 on the vast, desert-like, eastern arr..."
4,The Commuter,"a businessman, on hi daili commut home, get un..."


In [54]:
df_2018.shape

(266, 2)

In [55]:
link1 = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link1, header=0)[2]
df2 = pd.read_html(link1, header=0)[3]
df3 = pd.read_html(link1, header=0)[4]
df4 = pd.read_html(link1, header=0)[5]

In [56]:
df_2019 = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [57]:
df_2019.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]


In [58]:
df_2019.shape

(242, 6)

In [59]:
df_2019["genres"] = df_2019["Title"].map(lambda x: getGenre(str(x)))
df_2019["overview"] = df_2019["Title"].map(lambda x: getOverview(str(x)))

In [60]:
df_2019.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2],Thriller Action Mystery Horror,Six strangers find themselves in circumstances...
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3],Thriller Drama,When an overachieving college senior makes a w...
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4],Thriller,An unidentified man posts a live feed on socia...
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5],Drama Adventure Family,"The adventure of Bella, a dog who embarks on a..."
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6],Comedy Drama,Phillip is a wealthy quadriplegic who needs a ...


In [61]:
df_2019.isna().sum()

Opening                0
Opening.1              0
Title                  0
Production company     0
Cast and crew          0
Ref.                  10
genres                 0
overview               0
dtype: int64

In [62]:
df_2019["director"] = df_2019["Cast and crew"].apply(lambda x : getDirector_(x))
df_2019["actors"] = df_2019["Cast and crew"].apply(lambda x : getActors(x))

In [63]:
df_2019.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres,overview,director,actors
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2],Thriller Action Mystery Horror,Six strangers find themselves in circumstances...,[Adam Robitel],"[ Taylor Russell, Logan Miller, Deborah Ann ..."
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3],Thriller Drama,When an overachieving college senior makes a w...,[Jen McGowan],"[ Hermione Corfield, Jay Paulson, Sean O'Bryan]"
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4],Thriller,An unidentified man posts a live feed on socia...,[Wilson Coneybeare],"[ Donald Sutherland, Vincent Kartheiser, Oli..."
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5],Drama Adventure Family,"The adventure of Bella, a dog who embarks on a...",[Charles Martin Smith],"[ Bryce Dallas Howard, Edward James Olmos, A..."
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6],Comedy Drama,Phillip is a wealthy quadriplegic who needs a ...,[Neil Burger],"[ Bryan Cranston, Kevin Hart, Nicole Kidman]"


In [64]:
df_2019 = df_2019[["Title", "genres", "overview", "director", "actors"]]
df_2019["genres"] = df_2019["genres"].apply(lambda x : x.split())
df_2019["overview"] = df_2019["overview"].apply(lambda x : x.split())

In [65]:
df_2019.head()

,Title,genres,overview,director,actors
0,Escape Room,"[Thriller, Action, Mystery, Horror]","[Six, strangers, find, themselves, in, circums...",[Adam Robitel],"[ Taylor Russell, Logan Miller, Deborah Ann ..."
1,Rust Creek,"[Thriller, Drama]","[When, an, overachieving, college, senior, mak...",[Jen McGowan],"[ Hermione Corfield, Jay Paulson, Sean O'Bryan]"
2,American Hangman,[Thriller],"[An, unidentified, man, posts, a, live, feed, ...",[Wilson Coneybeare],"[ Donald Sutherland, Vincent Kartheiser, Oli..."
3,A Dog's Way Home,"[Drama, Adventure, Family]","[The, adventure, of, Bella,, a, dog, who, emba...",[Charles Martin Smith],"[ Bryce Dallas Howard, Edward James Olmos, A..."
4,The Upside,"[Comedy, Drama]","[Phillip, is, a, wealthy, quadriplegic, who, n...",[Neil Burger],"[ Bryan Cranston, Kevin Hart, Nicole Kidman]"


In [66]:
df_2019["Tags"] = df_2019["overview"] + df_2019["genres"] + df_2019["actors"] + df_2019["director"]

In [67]:
df_2019 = df_2019[["Title", "Tags"]]
df_2019["Tags"] = df_2019["Tags"].apply(lambda x : " ".join(x))
df_2019.head()

,Title,Tags
0,Escape Room,Six strangers find themselves in circumstances...
1,Rust Creek,When an overachieving college senior makes a w...
2,American Hangman,An unidentified man posts a live feed on socia...
3,A Dog's Way Home,"The adventure of Bella, a dog who embarks on a..."
4,The Upside,Phillip is a wealthy quadriplegic who needs a ...


In [68]:
df_2019["Tags"] = df_2019["Tags"].apply(stem)
df_2019["Tags"] = df_2019["Tags"].apply(lambda x : x.lower())

In [69]:
df_2019.head()

,Title,Tags
0,Escape Room,six stranger find themselv in circumst beyond ...
1,Rust Creek,when an overachiev colleg senior make a wrong ...
2,American Hangman,an unidentifi man post a live feed on social m...
3,A Dog's Way Home,"the adventur of bella, a dog who embark on an ..."
4,The Upside,phillip is a wealthi quadripleg who need a car...


In [70]:
df_2019.shape

(242, 2)

In [71]:
df_2017.shape, df_2018.shape, df_2019.shape

((242, 2), (266, 2), (242, 2))

In [72]:
finalData = pd.concat([df_2017, df_2018, df_2019])
finalData.shape

(750, 2)

In [73]:
finalData.head()

,Title,Tags
0,Underworld: Blood Wars,vampir death dealer selen fend off brutal atta...
1,Arsenal,after the deadbeat brother of a businessman is...
2,Between Us,"strand after a tragic plane crash, two strange..."
3,Monster Trucks,tripp is a high school senior with a knack for...
4,The Bye Bye Man,when three colleg student move into an old hou...


In [75]:
finalData.tail()

,Title,Tags
237,Spies in Disguise,super spi lanc sterl and scientist walter beck...
238,Little Women,four sister come of age in america in the afte...
239,1917,"at the height of the first world war, two youn..."
240,Just Mercy,the power true stori of harvard-educ lawyer br...
241,Clemency,year of carri out death row execut have taken ...


In [76]:
finalData.to_csv("../processedData/latestMovies.csv")